<a href="https://colab.research.google.com/github/bartoszpankratz/VAR_neural_network/blob/master/VaR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import scipy.stats as sps
import pandas as pd
import numpy as np

class VaR:
    def __init__(self, data, params):
        self.name     = data.name
        self.ticker   = data.ticker
        self.vars     = pd.DataFrame({
            'Date'    : data.data.index,
            'Returns' : data.data['Returns']
        })
        self.vars = self.vars.set_index('Date')
        self.params = params
        
        #if data.name in ['GARCH', 'GARCH_ARCH', 'GARCH_ARCH_T']:
        #    for i in range(len(params['ALPHA'])):
        #        name = ['TrueVaR_' + str(params['ALPHA'][i])]
        #        self.vars[name] = data.data[name]
        
        #self.backtest = {
        #    'Kupiec'         : [],
        #    'Bernoulli'      : [],
        #    'Christoffersen' : []
        #}
        
    def calc_VCA(self):
        params     = self.params
        look_back  = params['LOOK_BACK']
        INV_NORM   = sps.norm.ppf(params['ALPHA'])
        
        var_titles = ['VCA_' + str(params['ALPHA'][i]) for i in range(len(params['ALPHA']))]
        y     = self.vars['Returns']
        y_hat = []
        ind   = []
        
        for i in range(look_back, len(y)):
            x = y[i-look_back: i]
            y_hat.append(np.mean(x) + np.std(x) * INV_NORM)
            ind.append(self.vars.index.values[i])
            
        res = pd.DataFrame(y_hat, columns  = var_titles).set_index([ind])
        for name in var_titles:
            self.vars[name] = pd.Series(res[name].values, index=res.index)     
            
    def calc_Hist(self):
        params     = self.params
        look_back  = params['LOOK_BACK']
        
        var_titles = ['HIST_' + str(params['ALPHA'][i]) for i in range(len(params['ALPHA']))]
        y     = self.vars['Returns']
        y_hat = []
        ind   = []
        
        for i in range(look_back, len(y)):
            x = y[i-look_back: i]
            y_hat.append(np.percentile(x, q = np.array(params['ALPHA'])*100))
            ind.append(self.vars.index.values[i])
            
        res = pd.DataFrame(y_hat, columns  = var_titles).set_index([ind])
        for name in var_titles:
            self.vars[name] = pd.Series(res[name].values, index=res.index)